In [21]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import seaborn as sns
from torch.autograd import Variable
import cv2
import os

import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
# root_dir = os.getcwd()
base_dir = ""
# print(root_dir)

In [29]:
data_raw = pd.read_csv(base_dir + 'sign-language-mnist/sign_mnist_train.csv', sep=",")
test_data_raw = pd.read_csv(base_dir + 'sign-language-mnist/sign_mnist_test.csv', sep=",")

In [30]:
labels = data_raw['label']
data_raw.drop('label', axis=1, inplace=True)
labels_test = test_data_raw['label']
test_data_raw.drop('label', axis=1, inplace=True)

In [31]:
data = data_raw.values
labels = labels.values

test_data = test_data_raw.values
labels_test = labels_test.values

In [32]:
reshaped = []
for i in data:
    reshaped.append(i.reshape(1, 28, 28))
data = np.array(reshaped)

reshaped_test = []
for i in test_data:
    reshaped_test.append(i.reshape(1,28,28))
test_data = np.array(reshaped_test)

In [33]:
x = torch.FloatTensor(data)
y = torch.LongTensor(labels.tolist())

test_x = torch.FloatTensor(test_data)
test_y = torch.LongTensor(labels_test.tolist())

In [34]:
class Network(nn.Module): 
    
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3)
        self.pool1 = nn.MaxPool2d(2)
        
        self.conv2 = nn.Conv2d(10, 20, 3)
        self.pool2 = nn.MaxPool2d(2)
        
        self.conv3 = nn.Conv2d(20, 30, 3) 
        self.dropout1 = nn.Dropout2d()
        
        self.fc3 = nn.Linear(30 * 3 * 3, 270) 
        self.fc4 = nn.Linear(270, 26)
        
        self.softmax = nn.LogSoftmax(dim=1)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout1(x)
                
        x = x.view(-1, 30 * 3 * 3) 
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        
        return self.softmax(x)
    
    def test(self, predictions, labels):
        
        self.eval()
        correct = 0
        for p, l in zip(predictions, labels):
            if p == l:
                correct += 1
        
        acc = correct / len(predictions)
        print("Correct predictions: %5d / %5d (%5f)" % (correct, len(predictions), acc))
        
    def evaluate(self, predictions, labels):
                
        correct = 0
        for p, l in zip(predictions, labels):
            if p == l:
                correct += 1
        
        acc = correct / len(predictions)
        return(acc)
    WIDTH = 28
    HEIGHT = 28
    imageArray = []
    def predict(image):

        imageArray.append(cv2.resize(image, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC))
        imageNumArray = np.array(imageArray)
        image_tensor = torch.FloatTensor(imageNumArray)
        image_prediction = net(Variable(image_tensor))
        return(image_prediction.data[1][0])


In [35]:
!pip install torchsummary
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
model = Network().to(device)
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
         MaxPool2d-2           [-1, 10, 13, 13]               0
            Conv2d-3           [-1, 20, 11, 11]           1,820
         MaxPool2d-4             [-1, 20, 5, 5]               0
            Conv2d-5             [-1, 30, 3, 3]           5,430
         Dropout2d-6             [-1, 30, 3, 3]               0
            Linear-7                  [-1, 270]          73,170
            Linear-8                   [-1, 26]           7,046
        LogSoftmax-9                   [-1, 26]               0
Total params: 87,566
Trainable params: 87,566
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.09
Params size (MB): 0.33
Estimated Total Size (MB): 0.43
---------------------------------------------

In [37]:
net = Network()

optimizer = optim.SGD(net.parameters(),0.001, momentum=0.7)
loss_func = nn.CrossEntropyLoss()

In [ ]:
loss_log = []
acc_log = []

for e in range(50):
    for i in range(0, x.shape[0], 100):
        x_mini = x[i:i + 100] 
        y_mini = y[i:i + 100] 
        
        optimizer.zero_grad()
        net_out = net(Variable(x_mini))
        
        loss = loss_func(net_out, Variable(y_mini))
        loss.backward()
        optimizer.step()
        
        if i % 1000 == 0:
            #pred = net(Variable(test_data_formated))
            loss_log.append(loss.item())
            acc_log.append(net.evaluate(torch.max(net(Variable(test_x[:500])).data, 1)[1], test_y[:500]))
        
    print('Epoch: {} - Loss: {:.6f}'.format(e + 1, loss.item()))

Epoch: 1 - Loss: 2.711518
Epoch: 2 - Loss: 1.667976
Epoch: 3 - Loss: 1.240876
Epoch: 4 - Loss: 1.075021
Epoch: 5 - Loss: 0.848511
Epoch: 6 - Loss: 0.840606
Epoch: 7 - Loss: 0.546788
Epoch: 8 - Loss: 0.684898
Epoch: 9 - Loss: 0.422596
Epoch: 10 - Loss: 0.388091
Epoch: 11 - Loss: 0.408112
Epoch: 12 - Loss: 0.278472
Epoch: 13 - Loss: 0.247012
Epoch: 14 - Loss: 0.095179
Epoch: 15 - Loss: 0.241266
Epoch: 16 - Loss: 0.202471
Epoch: 17 - Loss: 0.250477
Epoch: 18 - Loss: 0.171151
Epoch: 19 - Loss: 0.192299
Epoch: 20 - Loss: 0.066520
Epoch: 21 - Loss: 0.153754
Epoch: 22 - Loss: 0.159042
Epoch: 23 - Loss: 0.090139
Epoch: 24 - Loss: 0.118866
Epoch: 25 - Loss: 0.108710
Epoch: 26 - Loss: 0.093142
Epoch: 27 - Loss: 0.067619
Epoch: 28 - Loss: 0.127031
Epoch: 29 - Loss: 0.177465
Epoch: 30 - Loss: 0.084559
Epoch: 31 - Loss: 0.012223
Epoch: 32 - Loss: 0.069902
Epoch: 33 - Loss: 0.173416
Epoch: 34 - Loss: 0.054696
Epoch: 35 - Loss: 0.092476


In [11]:
plt.figure(figsize=(10,8))
plt.plot(loss_log[2:])
plt.plot(acc_log)
plt.plot(np.ones(len(acc_log)), linestyle='dashed')
plt.show()

NameError: name 'plt' is not defined

In [12]:
predictions = net(Variable(test_x))
print(predictions)
net.test(torch.max(predictions.data, 1)[1], test_y)

NameError: name 'Variable' is not defined

In [13]:
print(torch.max(predictions.data, 1)[1][2])
pixels = data[2].reshape(28, 28)
plt.subplot(221)
sns.heatmap(data=pixels)

NameError: name 'predictions' is not defined